In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
%matplotlib inline

In [2]:
import sys
sys.path.append('/home/albert/github/DenseNet/')
import densenet
sys.path.append('/home/albert/github/tensorflow/src/')
import triplet
import evaluation
import data
import models

Using TensorFlow backend.


In [3]:
def evaluate_rank(all_embeddings, query_embeddings,
                    test_dict=None, test_files=None, query_files=None,
                    rank=[1,5], DATA_ROOT='/home/albert/github/tensorflow/'):
    if test_dict is None or test_files is None or query_files is None:
        test_dict, test_files = get_data('test', DATA_ROOT)
        _, query_files = get_data('query', DATA_ROOT)

    all_identities = np.array([p[1] for p in test_files])
    all_camera = np.array([p[2] for p in test_files])

    correct = np.array([0] * len(rank))
    test_iter = np.array([0] * len(rank))

    for q in range(len(query_files)):
        idt, camera = int(query_files[q][1]), int(query_files[q][2])
        b = np.logical_or(all_camera != camera, all_identities != idt)

        if len(test_dict[idt].keys()) > 1:
            query_embedding = query_embeddings[q]
            distance_vectors = np.squeeze(np.abs(all_embeddings[b] - query_embedding))
            distance = np.sum(distance_vectors, axis=1)

            for r in range(len(rank)):
                top_inds = distance.argsort()[:rank[r]]
                output_classes = all_identities[b][top_inds]
                # print output_classes

                if np.where(output_classes == idt)[0].shape[0] > 0:
                    correct[r] += 1
                test_iter[r] += 1

    return np.divide(correct.astype(np.float32), test_iter).tolist()